In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import joblib
from sklearn import metrics

Using TensorFlow backend.


In [2]:
data = pd.read_csv("../dataset")
data.count()

protocol          17106
transfer_rate     17106
avg_pkt_size      17106
host_ratio        17106
host_num          17106
diff_port_num     17106
diff_sport_num    17106
opp_ratio         17106
label             17106
dtype: int64

In [3]:
data.label.value_counts()

normal         3414
tcpsynflood    3172
portscan       2587
udpflood       2462
icmpflood      2224
ipsweep        2007
pingofdeath    1240
Name: label, dtype: int64

In [4]:
data.label.value_counts(normalize=True)

normal         0.199579
tcpsynflood    0.185432
portscan       0.151233
udpflood       0.143926
icmpflood      0.130013
ipsweep        0.117327
pingofdeath    0.072489
Name: label, dtype: float64

In [5]:
x_columns = data.columns.drop('label')
x = data[x_columns]
x = x.values
y = data['label']
x.shape

(17106, 8)

In [6]:
le_proto = LabelEncoder()
x[:, 0] = le_proto.fit_transform(x[:, 0])
joblib.dump(le_proto, '../preprocess/le_proto-conv2d.joblib')

le_label = LabelEncoder()
y = le_label.fit_transform(y)
joblib.dump(le_label, '../preprocess/le_label-conv2d.joblib')

sc = StandardScaler()
x=sc.fit_transform(x)

joblib.dump(sc, '../preprocess/scaler-conv2d.joblib')

x = x.reshape(x.shape[0],2,4,1)
x.shape

(17106, 2, 4, 1)

In [7]:
model = Sequential()

model.add(Conv2D(16, (2,2),padding = 'same', activation ='relu',input_shape = (2,4,1)))
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Conv2D(32, (2,2),padding = 'same',activation ='relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
model.add(Dense(64, activation = 'relu'))
model.add(Flatten())
model.add(Dense(7, activation = 'softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2, 4, 16)          80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 2, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 2, 32)          2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1, 64)          2112      
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                

In [8]:
save_checkpoint = ModelCheckpoint(
    filepath = '../trained_models/cnn2d.h5',
    monitor = 'val_accuracy',
    save_best_only = True,
    save_weights_only = False,
    mode = 'auto'
)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

history = model.fit(x, y, batch_size=64, validation_split=0.3,callbacks=[save_checkpoint], epochs=500,shuffle=False)

Train on 11974 samples, validate on 5132 samples
Epoch 1/500
11974/11974 [==============================] - 0s 38us/step - loss: 1.0910 - accuracy: 0.6580 - val_loss: 0.3779 - val_accuracy: 0.9172
Epoch 2/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.2232 - accuracy: 0.9471 - val_loss: 0.1595 - val_accuracy: 0.9649
Epoch 3/500
11974/11974 [==============================] - 0s 27us/step - loss: 0.1306 - accuracy: 0.9620 - val_loss: 0.1172 - val_accuracy: 0.9745
Epoch 4/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.1016 - accuracy: 0.9685 - val_loss: 0.0991 - val_accuracy: 0.9766
Epoch 5/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0877 - accuracy: 0.9711 - val_loss: 0.0886 - val_accuracy: 0.9782
Epoch 6/500
11974/11974 [==============================] - 0s 29us/step - loss: 0.0797 - accuracy: 0.9734 - val_loss: 0.0821 - val_accuracy: 0.9790
Epoch 7/500
11974/11974 [==============================] - 0s 2

Epoch 56/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0362 - accuracy: 0.9852 - val_loss: 0.0512 - val_accuracy: 0.9838
Epoch 57/500
11974/11974 [==============================] - 0s 22us/step - loss: 0.0360 - accuracy: 0.9853 - val_loss: 0.0506 - val_accuracy: 0.9852
Epoch 58/500
11974/11974 [==============================] - 0s 23us/step - loss: 0.0359 - accuracy: 0.9854 - val_loss: 0.0505 - val_accuracy: 0.9856
Epoch 59/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0356 - accuracy: 0.9856 - val_loss: 0.0501 - val_accuracy: 0.9852
Epoch 60/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0356 - accuracy: 0.9856 - val_loss: 0.0513 - val_accuracy: 0.9836
Epoch 61/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0354 - accuracy: 0.9857 - val_loss: 0.0509 - val_accuracy: 0.9850
Epoch 62/500
11974/11974 [==============================] - 0s 23us/step - loss: 0.0354 - accuracy: 0.9856

Epoch 111/500
11974/11974 [==============================] - 0s 23us/step - loss: 0.0311 - accuracy: 0.9866 - val_loss: 0.0472 - val_accuracy: 0.9846
Epoch 112/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0308 - accuracy: 0.9869 - val_loss: 0.0474 - val_accuracy: 0.9848
Epoch 113/500
11974/11974 [==============================] - 0s 32us/step - loss: 0.0308 - accuracy: 0.9868 - val_loss: 0.0472 - val_accuracy: 0.9848
Epoch 114/500
11974/11974 [==============================] - 0s 22us/step - loss: 0.0307 - accuracy: 0.9865 - val_loss: 0.0475 - val_accuracy: 0.9840
Epoch 115/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0308 - accuracy: 0.9864 - val_loss: 0.0476 - val_accuracy: 0.9844
Epoch 116/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0304 - accuracy: 0.9869 - val_loss: 0.0473 - val_accuracy: 0.9848
Epoch 117/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0306 - accuracy:

Epoch 220/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0240 - accuracy: 0.9889 - val_loss: 0.0497 - val_accuracy: 0.9850
Epoch 221/500
11974/11974 [==============================] - 0s 22us/step - loss: 0.0243 - accuracy: 0.9890 - val_loss: 0.0500 - val_accuracy: 0.9848
Epoch 222/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0242 - accuracy: 0.9885 - val_loss: 0.0504 - val_accuracy: 0.9850
Epoch 223/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0240 - accuracy: 0.9890 - val_loss: 0.0504 - val_accuracy: 0.9848
Epoch 224/500
11974/11974 [==============================] - 0s 28us/step - loss: 0.0238 - accuracy: 0.9890 - val_loss: 0.0501 - val_accuracy: 0.9852
Epoch 225/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0238 - accuracy: 0.9894 - val_loss: 0.0497 - val_accuracy: 0.9852
Epoch 226/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0238 - accuracy:

Epoch 329/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0198 - accuracy: 0.9908 - val_loss: 0.0529 - val_accuracy: 0.9854
Epoch 330/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0198 - accuracy: 0.9911 - val_loss: 0.0540 - val_accuracy: 0.9856
Epoch 331/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0199 - accuracy: 0.9911 - val_loss: 0.0549 - val_accuracy: 0.9852
Epoch 332/500
11974/11974 [==============================] - 0s 29us/step - loss: 0.0198 - accuracy: 0.9910 - val_loss: 0.0531 - val_accuracy: 0.9852
Epoch 333/500
11974/11974 [==============================] - 0s 27us/step - loss: 0.0205 - accuracy: 0.9908 - val_loss: 0.0535 - val_accuracy: 0.9856
Epoch 334/500
11974/11974 [==============================] - 0s 26us/step - loss: 0.0197 - accuracy: 0.9907 - val_loss: 0.0532 - val_accuracy: 0.9854
Epoch 335/500
11974/11974 [==============================] - 0s 33us/step - loss: 0.0193 - accuracy:

Epoch 438/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0177 - accuracy: 0.9921 - val_loss: 0.0532 - val_accuracy: 0.9867
Epoch 439/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0179 - accuracy: 0.9924 - val_loss: 0.0514 - val_accuracy: 0.9869
Epoch 440/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0175 - accuracy: 0.9916 - val_loss: 0.0539 - val_accuracy: 0.9864
Epoch 441/500
11974/11974 [==============================] - 0s 25us/step - loss: 0.0182 - accuracy: 0.9922 - val_loss: 0.0517 - val_accuracy: 0.9875
Epoch 442/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0185 - accuracy: 0.9921 - val_loss: 0.0544 - val_accuracy: 0.9866
Epoch 443/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0174 - accuracy: 0.9921 - val_loss: 0.0534 - val_accuracy: 0.9869
Epoch 444/500
11974/11974 [==============================] - 0s 24us/step - loss: 0.0179 - accuracy:

In [9]:
# import matplotlib.pyplot as plt
# import matplotlib

# matplotlib.rcParams.update({'font.size': 20})
# plt.figure(figsize=(20,16))
# plt.plot(history.history['val_accuracy'], color='b', label="Validation Accuracy")
# plt.plot(history.history['accuracy'], color='r', label="Training Accuracy")

# plt.ylim([0.9,1])
# plt.xlim([0,len(history.history['loss'])])
# plt.title('Model accuracy')

# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Validation', 'Train'])
# plt.savefig('cnn2d-acc.png')
# plt.show()


In [10]:
# plt.figure(figsize=(20,16))
# plt.plot(history.history['val_loss'], color='b', label="Validation Loss")
# plt.plot(history.history['loss'], color='r', label="Training Loss")

# plt.ylim([0,0.6])
# plt.xlim([0,len(history.history['loss'])])
# plt.title('Model loss')

# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Validation', 'Train'], loc='upper left')
# plt.savefig('cnn2d-loss.png')
# plt.show()

In [11]:
max(history.history['val_accuracy'])

0.9888932108879089

In [12]:
model = load_model('../trained_models/cnn2d.h5')

datatest = pd.read_csv("../datatest")
x_columns = data.columns.drop('label')
x = data[x_columns]
x = x.values
y = data['label']

x[:, 0] = le_proto.transform(x[:, 0])
#joblib.dump(le_proto, '../preproc/le_proto2.joblib',protocol=1) #save label encoder weights to file
y = le_label.transform(y)
#joblib.dump(le_label, '../preproc/le_label2.joblib',protocol=1) #save label encoder weights to file
x=sc.transform(x)
#joblib.dump(sc, '../preproc/scaler2.joblib',protocol=1)
x = x.reshape(x.shape[0],2,4,1)

y_pred = model.predict_classes(x)
y_pred = le_label.inverse_transform(y_pred)
y_test = le_label.inverse_transform(y)
stats = {}
labels = ['normal','tcpsynflood','icmpflood','udpflood','ipsweep','portscan','pingofdeath']
for i in labels:
    stats[i]={}
    for j in labels:
        stats[i][j] = 0

In [13]:
for i in range(len(y)):
    stats[y_test[i]][y_pred[i]] += 1
stats

{'normal': {'normal': 3394,
  'tcpsynflood': 8,
  'icmpflood': 5,
  'udpflood': 0,
  'ipsweep': 5,
  'portscan': 2,
  'pingofdeath': 0},
 'tcpsynflood': {'normal': 0,
  'tcpsynflood': 3172,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'icmpflood': {'normal': 116,
  'tcpsynflood': 0,
  'icmpflood': 2108,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'udpflood': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 2462,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'ipsweep': {'normal': 1,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 2006,
  'portscan': 0,
  'pingofdeath': 0},
 'portscan': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 2587,
  'pingofdeath': 0},
 'pingofdeath': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 1,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 1239}}

In [14]:
tpr={}
fpr={}
precision={}
f1score={}
for i in labels:
    tp = stats[i][i]
    fp = 0
    tn = 0
    fn = 0
    for j in labels:
        if i != j:
            fn += stats[i][j]
            fp += stats[j][i]
            for k in stats[j].keys():
                if k != i:
                    tn += stats[j][k]
    tpr[i] = tp / (tp + fn)
    fpr[i] = fp / (fp + tn)
    precision[i] = tp / (tp + fp)
    f1score[i] = 2/(1/tpr[i] + 1/precision[i])
print('TPR: \n ' + str(tpr))
print('FRP:\n ' + str(fpr))
print('Precision:\n ' + str(precision))
print('F1-score: \n' + str(f1score))

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

TPR: 
 {'normal': 0.994141769185706, 'tcpsynflood': 1.0, 'icmpflood': 0.947841726618705, 'udpflood': 1.0, 'ipsweep': 0.9995017438963627, 'portscan': 1.0, 'pingofdeath': 0.9991935483870967}
FRP:
 {'normal': 0.008545135845749343, 'tcpsynflood': 0.0005741352088416823, 'icmpflood': 0.00040317161671818305, 'udpflood': 0.0, 'ipsweep': 0.0003311477581296775, 'portscan': 0.00013775053378331842, 'pingofdeath': 0.0}
Precision:
 {'normal': 0.9666761606379949, 'tcpsynflood': 0.9974842767295597, 'icmpflood': 0.9971617786187322, 'udpflood': 1.0, 'ipsweep': 0.9975136747886624, 'portscan': 0.9992275009656237, 'pingofdeath': 1.0}
F1-score: 
{'normal': 0.980216606498195, 'tcpsynflood': 0.9987405541561712, 'icmpflood': 0.9718764407561089, 'udpflood': 1.0, 'ipsweep': 0.998506719761075, 'portscan': 0.999613601236476, 'pingofdeath': 0.9995966115369102}
Accuracy: 0.9919326552086987
